In [1]:
%pip install langchain langchain_openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/5

In [2]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import Field, BaseModel
from typing import List

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [5]:
pip install --upgrade pydantic


In [6]:
from pydantic import BaseModel, Field


In [7]:
class ArticleSectionOutline(BaseModel):
    title: str = Field(description="The title of the section")

class ArticleOutline(BaseModel):
    title: str = Field(description="The title of the article")
    sections: List[ArticleSectionOutline] = Field(description="All of the sections of the article")

parser = PydanticOutputParser(pydantic_object=ArticleOutline)
# For Pydantic v1, use `schema()` instead of `model_json_schema()`
format_instructions = parser.pydantic_object.schema()

llm: ChatOpenAI = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    temperature=0,
    api_key="not-needed",
    model_kwargs={"stop": ['```']},
)

article_plan = llm.invoke(
    [
        SystemMessage(
            content=f"""Act as a SEO specialist.
                  You are very specialised in generating highly optimized SEO briefs for content marketers.
                  You must respond in JSON format only
                  {format_instructions}

                  You must finish it with }} ```
                  """
        ),
        HumanMessage(
            content="""I want you to generate an effective article plan for me on Digital Marketing
            \n```"""
        ),
    ]
)


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: Parameters {'stop'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if (await self.run_code(code, result,  async_=asy)):


APIConnectionError: Connection error.

In [4]:
# Article Outline using Pydantic Models:
class ArticleSectionOutline(BaseModel):
    title: str = Field(description="The title of the section")

class ArticleOutline(BaseModel):
    title: str = Field(description="The title of the article")
    sections: List[ArticleSectionOutline] = Field(description="All of the sections of the article")

parser = PydanticOutputParser(pydantic_object=ArticleOutline)
format_instructions = parser.get_format_instructions()

llm: ChatOpenAI = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    temperature=0,
    api_key="not-needed",
    model_kwargs={"stop": ['```']},
)

article_plan = llm.invoke(
    [
        SystemMessage(
            content=f"""Act as a SEO specialist.
                  You are very specialised in generating highly optimized SEO briefs for content marketers.
                  You must respond in JSON format only
                  {format_instructions}

                  You must finish it with }} ```
                  """
        ),
        HumanMessage(
            content="""I want you to generate an effective article plan for me on Digital Marketing
            \n```"""
        ),
    ]
)

AttributeError: type object 'ArticleOutline' has no attribute 'model_json_schema'

In [ ]:
article_outline = parser.parse(article_plan.content)

In [ ]:
print(article_outline.sections)